# TODO Automate this for all NCAR files. Now it works for just 1st file

<font color="red">Dask everything</font>

## Convert all netCDF NCAR timestep files to Zarr 512 arrays, with Grouped Velocity components, with (64,64,64) chunk size, round-robined across FileDB nodes (spatially using Z-order)

In [4]:
desired_cube_side = 512
chunk_size = 64
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF'
# Has issues on SciServer Compute
use_dask = True
dest_folder_name = "sabl2048a"
timestep_nr = 0

In [ ]:
!pip install "dask[complete]"
!pip install "xarray[complete]"
!pip install morton-py

In [1]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [2]:
import xarray as xr
import os
import write_tools

<font color="orange">Don't delete the CD cell!</font>

In [4]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF

/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF


In [5]:
data_xr = xr.open_dataset("/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF/jhd.00" + str(timestep_nr) + ".nc")
# data_xr.info

## Group 3 velocity components together

<font color="red">Old Dask version gives this error https://github.com/dask/distributed/issues/3955</font>

In [ ]:
# # For Testing purposes - make data much smaller - faster

# desired_cube_side = 64
# smaller_size = 128
# i=0
# j=0
# k=0
# dims = [dim for dim in data_xr.dims]

# data_xr = data_xr.isel( # TODO make this into a function: slice_group()
#     {dims[0]: slice(i * smaller_size, (i + 1) * smaller_size),
#      dims[1]: slice(j * smaller_size, (j + 1) * smaller_size),
#      dims[2]: slice(k * smaller_size, (k + 1) * smaller_size)}
# )

In [ ]:
merged_velocity = write_tools.merge_velocities(data_xr, chunk_size_base=chunk_size, use_dask=use_dask)

## Unabbreviate 'e', 'p', 't' variable names

In [ ]:
merged_velocity = merged_velocity.rename({'e': 'energy', 't': 'temperature', 'p': 'pressure'})

In [ ]:
# merged_velocity

### Split 2048^3 into smaller 512^3 arrays

In [ ]:
dims = [dim for dim in data_xr.dims]
smaller_groups = write_tools.split_zarr_group(merged_velocity, desired_cube_side, dims)

In [ ]:
# smaller_groups should be a cube (list of lists of lists)
# smaller_groups[0][0][0]

## Z-order the smaller Arrays

In [11]:
z_order = write_tools.morton_order_cubes(smaller_groups)
# z_order

[[[0, 4], [2, 6]], [[1, 5], [3, 7]]]

## Distribute them across FileDB

### Get target Folder list

In [5]:
folders=write_tools.list_fileDB_folders()

# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/") # This is already created

for i in range(len(folders)):
    folders[i] += dest_folder_name + "_" + str(i + 1).zfill(2) + "_prod"

folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/sabl2048a_01_prod',
 '/home/idies/workspace/turb/data02_01/zarr/sabl2048a_02_prod',
 '/home/idies/workspace/turb/data03_01/zarr/sabl2048a_03_prod',
 '/home/idies/workspace/turb/data04_01/zarr/sabl2048a_04_prod',
 '/home/idies/workspace/turb/data05_01/zarr/sabl2048a_05_prod']

In [22]:
# Make top-level dir

# for folder_path in folders:
#     os.makedirs(folder_path, exist_ok=False)

In [20]:
import dask

cubes = smaller_groups


@dask.delayed
def write_to_disk(z_order_nr, current_array, filedb_index):
    encoding={
        "velocity": dict(chunks=(chunk_size, chunk_size, chunk_size, 3), compressor=None),
        "pressure": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None),
        "temperature": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None),
        "energy": dict(chunks=(chunk_size, chunk_size, chunk_size, 1), compressor=None)
    }
#     encoding = {variable_name: {'compressor': None, 'chunks': (chunk_size, chunk_size, chunk_size)} for variable_name in current_array.variables}
    
    dest_groupname = folders[filedb_index] + dest_folder_name + str(z_order_nr).zfill(2) + "_" + str(timestep_nr).zfill(2) + ".zarr"
    
    current_array.to_zarr(store=dest_groupname,
        mode="w",
        encoding = encoding)


tasks = []
for i in range(len(cubes)):
    for j in range(len(cubes[i])):
        for k in range(len(cubes[i][j])):
            # Put e.g. z-order=4 to filedb05_01
            filedb_index = z_order[i][j][k] % len(folders) # Loop around
            current_array = cubes[i][j][k]
            if use_dask:
                tasks.append(write_to_disk(z_order[i][j][k], current_array, filedb_index))
            else:
                write_to_disk(z_order[i][j][k], current_array, filedb_index)

if use_dask:
    dask.compute(*tasks)

(None, None, None, None, None, None, None, None)

In [ ]:
# i = 0

# # Possible Parallel implementation issue:
# # Ariel: possible contention on data_xr = xr.open_dataset(file_name)
# #     bcs all source data live on data02_02
# for file_name in os.listdir(raw_ncar_folder_path):
#     if os.path.isfile(os.path.join(raw_ncar_folder_path, file_name)):
#         # https://github.com/pangeo-data/pangeo/issues/150
#         data_xr = xr.open_dataset(file_name)

#         # Disable compression, set chunk size
#         encoding = {variable_name: {'compressor': None, 'chunks': (chunk_size_base, chunk_size_base, chunk_size_base)} for variable_name in data_xr.variables}
        
#         target_dir = folders[i]

#         # overwrite if exists
#         data_xr.to_zarr(store=target_dir + "ncar_" + str(i) + "_" + "chunk_" + str(chunk_size_base) + ".zarr",
#                         mode="w",
#                        encoding = encoding)
        
#         print(file_name)
#         i += 1
